# ResNet on CIFAR-10
This notebook is used to experiment with ResNet-50 on CIFAR-10 dataset.

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torchvision as tv
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import sys

sys.path.append('../../')
sys.path.append('../../src/')

import src.general as general
import src.dataset_models as data
import src.metrics as metrics
import src.evaluation as eval
import src.plot as plot
import src.compression.distillation as distill
import src.compression.pruning as prune
import src.compression.quantization as quant


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Get dataset
dataset = data.supported_datasets["CIFAR-10"]

# Get transforms
resnet_cifar10_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Set transforms
dataset.set_transforms(resnet_cifar10_transform)

In [49]:
# Get model
resnet_weights = tv.models.ResNet50_Weights.DEFAULT
resnet = tv.models.resnet50(weights=resnet_weights)

# Modify the last layer to have 10 output classes (CIFAR-10 has 10 classes)
resnet.fc = torch.nn.Linear(resnet.fc.in_features, 10)

In [10]:
MODELS_DIR = "/workspace/volume/models"

In [5]:
resnet = torch.load("/workspace/volume/models/resnet_cifar_finetuned.pth")

In [11]:
torch.save(resnet, MODELS_DIR + "/resnet_cifar_finetuned.pth")

In [6]:
general.finetune(model=resnet, dataset=dataset, target=99, max_it=20)
torch.save(resnet, MODELS_DIR + "/resnet_cifar_finetuned.pth")

Train: 100%|██████████| 5625/5625 [04:59<00:00, 18.79it/s]


Train loss: 0.0778
Train score: 97.4333


Validate: 100%|██████████| 79/79 [00:11<00:00,  6.88it/s]


Test loss: 0.1508
Test score: 95.0356


Train: 100%|██████████| 5625/5625 [04:50<00:00, 19.35it/s]


...

Train loss: 0.0223
Train score: 99.3111


Validate: 100%|██████████| 79/79 [00:11<00:00,  6.86it/s]


Test loss: 0.1404
Test score: 95.9652


Train:   5%|▌         | 301/5625 [00:15<04:26, 19.96it/s]


KeyboardInterrupt: 

In [34]:
model = torch.load(MODELS_DIR + "/resnet_cifar10_full.pt")

In [8]:
before_results = eval.get_results(resnet, dataset)
plot.print_results(**before_results)

Test: 100%|██████████| 157/157 [00:23<00:00,  6.64it/s]


Test loss: 0.1749
Test score: 94.8248
Could not calculate FLOPS
==================================== RESULTS ===================================
Loss: 0.174857
Score: 94.824841
Time per data point: 9.4163 ms
Model Size: 90.04 MB
Number of parameters: 23528522
Number of MACs: 4119896576


## Pruning

In [ ]:
pruned_model = prune.magnitude_pruning_structured(resnet, dataset, sparsity=0.5, fineTune=True, iterative_steps=5)

In [ ]:
general.finetune(pruned_model, dataset, target=98, max_it=10)

In [ ]:
torch.save(resnet.state_dict(), "/workspace/volume/models/resnet_cifar10.pt")

In [ ]:
torch.save(pruned_model, "/workspace/volume/models/resnet_cifar_pruned.pt")

In [ ]:
after_results = eval.get_results(pruned_model, dataset)

In [ ]:
plot.print_before_after_results(before_results, after_results)

## Quantization

In [36]:
#Load basic model
model = torch.load("/workspace/volume/models/resnet_cifar10_full.pt")

In [38]:
before_results = eval.get_results(model, dataset)
plot.print_results(**before_results)

Test: 100%|██████████| 157/157 [00:17<00:00,  8.77it/s]


Test loss: 0.1841
Test score: 94.8348
Could not calculate FLOPS
==================================== RESULTS ===================================
Loss: 0.184128
Score: 94.834793
Time per data point: 7.1300 ms
Model Size: 90.04 MB
Number of parameters: 23528522
Number of MACs: 4119896576


In [41]:
quant.static_quantization(model, dataset, calibrate=False)

/usr/local/lib/python3.8/dist-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


RuntimeError: Unsupported qscheme: per_channel_affine

In [47]:
quant.fuse_modules(model)

AttributeError: 'Conv2d' object has no attribute 'split'

In [43]:
quant.get_modules_to_fuse(model)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


[[Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
  BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
  ReLU(inplace=True)],
 [Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False),
  BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)],
 [Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
  BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)],
 [Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False),
  BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
  ReLU(inplace=True)],
 [Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False),
  BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)],
 [Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False),
  BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)],
 [Conv2d(64, 64, kernel_si

In [27]:
general.validate(model, dataset)

Validate: 100%|██████████| 79/79 [00:08<00:00,  8.86it/s]

Test loss: 0.0126
Test score: 99.6638


(0.012633227394146644,
 99.66376582278481,
 8919.443130493164,
 112.90434342396411,
 14.113042927995513)